<a href="https://colab.research.google.com/github/garfield-gray/DataMining/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json

# Imperical

In [125]:
# Address of files
%cd /content/drive/MyDrive/Uni/Data/DataMining/Winter2024/CA#6
!ls

/content/drive/MyDrive/Uni/Data/DataMining/Winter2024/CA#6
CA6.pdf		 citing.txt	      musae_git_edges.csv      musae_git_target.csv
CA#6Report.gdoc  Deep_Learning.ipynb  musae_git_features.json  README.txt


In [126]:

json_file_path = 'musae_git_features.json'
with open(json_file_path) as f:
    data = json.load(f)



df1 = pd.DataFrame.from_dict(data, orient='index')

df1['node'] = df1.index
df1['node'] = df1['node'].astype(int)

melted_df = df1.melt(id_vars=['node'], var_name='feature_name', value_name='feature')


result_df = melted_df.drop(columns=['feature_name'])

features = result_df.sort_values(by=['node']).dropna()


In [127]:
target = pd.read_csv("musae_git_target.csv")

edges = pd.read_csv("musae_git_edges.csv")

In [128]:
features

,node,feature
0,0,1574.0
678600,0,1578.0
640900,0,2115.0
603200,0,3098.0
565500,0,2480.0
...,...,...
188499,37699,676.0
150799,37699,3437.0
113099,37699,1663.0
75399,37699,509.0


In [130]:
target

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1
...,...,...,...
37695,37695,shawnwanderson,1
37696,37696,kris-ipeh,0
37697,37697,qpautrat,0
37698,37698,Injabie3,1


In [131]:
edges

,id_1,id_2
0,0,23977
1,1,34526
2,1,2370
3,1,14683
4,1,29982
...,...,...
288998,37527,37596
288999,37529,37601
289000,37644,2347
289001,25879,2347
